<a href="https://colab.research.google.com/github/rebirthmagex/Jogo_Bagha_Chall/blob/main/point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from contextlib import suppress
from pprint import pprint

import math
from enum import Enum
from copy import deepcopy
from collections import deque

import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
class Point:
  """A point on the board"""

  # A1 = top left, E5 = bottom right
  # A1 = point[0], B1 = point[1], E5 = point[24]
  #   A   B   C   D   E
  # 1 T               T
  #   | \ | / | \ | / |
  # 2
  #   | / | \ | / | \ |
  # 3
  #   | \ | / | \ | / |
  # 4
  #   | / | \ | / | \ |
  # 5 T               T

  # Mapeamento de coordenadas para índices e vice-versa
  COORD_TO_INDEX = {
      'A1': 0, 'B1': 1, 'C1': 2, 'D1': 3, 'E1': 4,
      'A2': 5, 'B2': 6, 'C2': 7, 'D2': 8, 'E2': 9,
      'A3': 10, 'B3': 11, 'C3': 12, 'D3': 13, 'E3': 14,
      'A4': 15, 'B4': 16, 'C4': 17, 'D4': 18, 'E4': 19,
      'A5': 20, 'B5': 21, 'C5': 22, 'D5': 23, 'E5': 24,
  }

  INDEX_TO_COORD = {v: k for k, v in COORD_TO_INDEX.items()}

  # Mapeamento de coordenadas para coordenadas do plano cartesiano e vice-versa
  COORD_TO_CARTESIAN_PLANE = {
      'A1': (0, 0), 'B1': (0, 1), 'C1': (0, 2), 'D1': (0, 3), 'E1': (0, 4),
      'A2': (1, 0), 'B2': (1, 1), 'C2': (1, 2), 'D2': (1, 3), 'E2': (1, 4),
      'A3': (2, 0), 'B3': (2, 1), 'C3': (2, 2), 'D3': (2, 3), 'E3': (2, 4),
      'A4': (3, 0), 'B4': (3, 1), 'C4': (3, 2), 'D4': (3, 3), 'E4': (3, 4),
      'A5': (4, 0), 'B5': (4, 1), 'C5': (4, 2), 'D5': (4, 3), 'E5': (4, 4),
  }

  CARTESIAN_PLANE_TO_COORD = {v: k for k, v in COORD_TO_CARTESIAN_PLANE.items()}

  # Direções possíveis de movimento
  DIRECTIONS = [
      (-1, 0), (1, 0), (0, -1), (0, 1),  # Movimentos verticais e horizontais
      (-1, -1), (-1, 1), (1, -1), (1, 1)  # Movimentos diagonais
  ]

  # Definindo conexões de movimentos e capturas usando coordenadas
  _move_connections = {
      'A1': ['B1', 'A2', 'B2'],
      'B1': ['A1', 'C1', 'A2', 'B2', 'C2'],
      'C1': ['B1', 'D1', 'B2', 'C2', 'D2'],
      'D1': ['C1', 'E1', 'C2', 'D2', 'E2'],
      'E1': ['D1', 'E2', 'D2'],

      'A2': ['A1', 'A3', 'B2', 'B1', 'B3'],
      'B2': ['B1', 'B3', 'A2', 'C2', 'A1', 'C1', 'A3','C3'],
      'C2': ['C1', 'C3', 'B2', 'D2', 'B1', 'D1', 'B3','D3'],
      'D2': ['D1', 'D3', 'C2', 'E2', 'C1', 'E1', 'C3','E3'],
      'E2': ['E1', 'E3', 'D2', 'D1', 'D3'],

      'A3': ['A2', 'A4', 'B3', 'B2', 'B4'],
      'B3': ['B2', 'B4', 'A3', 'C3', 'A2', 'C2', 'A4','C4'],
      'C3': ['C2', 'C4', 'B3', 'D3', 'B2', 'D2', 'B4','D4'],
      'D3': ['D2', 'D4', 'C3', 'E3', 'C2', 'E2', 'C4','E4'],
      'E3': ['E2', 'E4', 'D3', 'D2', 'D4'],

      'A4': ['A3', 'A5', 'B4', 'B3', 'B5'],
      'B4': ['B3', 'B5', 'A4', 'C4', 'A3', 'C3', 'A5','C5'],
      'C4': ['C3', 'C5', 'B4', 'D4', 'B3', 'D3', 'B5','D5'],
      'D4': ['D3', 'D5', 'C4', 'E4', 'C3', 'E3', 'C5','E5'],
      'E4': ['E3', 'E5', 'D4'],

      'A5': ['A4', 'B5', 'B4'],
      'B5': ['B4', 'A5', 'C5', 'A4', 'C4'],
      'C5': ['C4', 'D5', 'B5', 'B4', 'D4'],
      'D5': ['D4', 'E5', 'C5', 'C4', 'E4'],
      'E5': ['D4','D5', 'E4'],
  }

  _capture_connections = {
      'A1': ['C1', 'A3', 'C3'],
      'B1': ['D1', 'B3', 'D3'],
      'C1': ['A1', 'E1', 'A3', 'C3', 'E3'],
      'D1': ['B1', 'D3', 'B3'],
      'E1': ['C1', 'E3', 'C3'],

      'A2': ['C2', 'A4', 'C4'],
      'B2': ['D2', 'B4', 'D4'],
      'C2': ['A2', 'E2', 'A4', 'C4', 'E4'],
      'D2': ['B2', 'D4', 'B4'],
      'E2': ['C2', 'E4', 'C4'],

      'A3': ['C3', 'A1', 'C1', 'A5', 'C1', 'C5'],
      'B3': ['D3', 'B1', 'D1', 'B5'],
      'C3': ['A3', 'E3', 'A1', 'C1', 'E1', 'A5', 'C5', 'E5'],
      'D3': ['B3', 'D5', 'B1', 'D1', 'B5'],
      'E3': ['C3', 'E5', 'C1', 'E1', 'C5'],

      'A4': ['C4', 'A2', 'C2'],
      'B4': ['D4', 'B2', 'D2', 'D4'],
      'C4': ['A4', 'E4', 'A2', 'C2', 'E2'],
      'D4': ['B2', 'B4', 'D2'],
      'E4': ['C4', 'E2', 'C2'],

      'A5': ['C5', 'A3', 'C3'],
      'B5': ['D5', 'B3', 'D3'],
      'C5': ['A5', 'E5', 'A3', 'C3', 'E3'],
      'D5': ['B5', 'D3', 'B3'],
      'E5': ['C5', 'E3', 'C3'],
  }

  _dead_connections = {
      'B1': ['A1', 'C1'],
      'C1': ['B1', 'D1'],
      'D1': ['C1', 'E1'],

      'A2': ['A1', 'A3'],
      'B2': ['A1', 'B1', 'C1', 'A2', 'C2', 'A3', 'B3', 'C3'],
      'C2': ['B1', 'C1', 'D1', 'B2', 'D2', 'B3', 'C3', 'D3'],
      'D2': ['C1', 'D1', 'E1', 'C2', 'E2', 'C3', 'D3', 'E3'],
      'E2': ['E1', 'E3'],

      'A3': ['A2', 'A4'],
      'B3': ['A2', 'B2', 'C2', 'A3', 'C3', 'A4', 'B4', 'C4'],
      'C3': ['B2', 'C2', 'D2', 'B3', 'D3', 'B4', 'C4', 'D4'],
      'D3': ['C2', 'D2', 'E2', 'C4', 'E4', 'C4', 'D4', 'E4'],
      'E3': ['E2', 'E4'],

      'A4': ['A3', 'A5'],
      'B4': ['A3', 'B3', 'C3', 'A4', 'C4', 'A5', 'B5', 'C5'],
      'C4': ['B3', 'C3', 'D3', 'B4', 'D4', 'B5', 'C5', 'D5'],
      'D4': ['C3', 'D3', 'E3', 'C4', 'E4', 'C5', 'D5', 'E5'],
      'E4': ['E3', 'E5'],

      'B5': ['A5', 'C5'],
      'C5': ['B5', 'C5'],
      'D5': ['C5', 'E5']
  }

  @classmethod
  def get_connections(cls, position):
      """
      Retorna as conexões de uma posição.
      """
      return cls._move_connections.get(position, []) + cls._capture_connections.get(position, [])

#Testes

In [13]:
#posições o tigre tem que estar para ameaçar a cabra
Point._dead_connections['A2']

['A1', 'A3']

In [14]:
#posições para as quais uma peça pode se mover
Point._move_connections['A2']

['A1', 'A3', 'B2', 'B1', 'B3']

In [15]:
#posições que um tigre pode pular para capturar uma cabra
Point._capture_connections['A2']

['C2', 'A4', 'C4']

In [16]:
#todas as posições que estão conectadas uma determinada posição.
#essas posições incluem as posições de movimento e de captura
Point.get_connections('A2')

['A1', 'A3', 'B2', 'B1', 'B3', 'C2', 'A4', 'C4']

In [24]:
#coordenada geográfica para posição do tabuleiro
Point.CARTESIAN_PLANE_TO_COORD[(1,0)]

'A2'

In [23]:
#posição do tabuleiro para coordenada geográfica
Point.COORD_TO_CARTESIAN_PLANE['A2']

(1, 0)